# Use Closed-Form Policy to Play LunarLanderContinuous-v2

In [1]:
import sys
import logging
import itertools

import numpy as np
np.random.seed(0)
import gym

logging.basicConfig(level=logging.INFO,
        format='%(asctime)s [%(levelname)s] %(message)s',
        stream=sys.stdout, datefmt='%H:%M:%S')

In [2]:
env = gym.make('LunarLanderContinuous-v2', new_step_api=True)
for key in vars(env):
    logging.info('%s: %s', key, vars(env)[key])
for key in vars(env.spec):
    logging.info('%s: %s', key, vars(env.spec)[key])

00:00:00 [INFO] env: <LunarLanderContinuous<LunarLanderContinuous-v2>>
00:00:00 [INFO] action_space: Box(-1.0, 1.0, (2,), float32)
00:00:00 [INFO] observation_space: Box(-inf, inf, (8,), float32)
00:00:00 [INFO] reward_range: (-inf, inf)
00:00:00 [INFO] metadata: {'render.modes': ['human', 'rgb_array'], 'video.frames_per_second': 50}
00:00:00 [INFO] _max_episode_steps: 1000
00:00:00 [INFO] _elapsed_steps: None
00:00:00 [INFO] id: LunarLanderContinuous-v2
00:00:00 [INFO] entry_point: gym.envs.box2d:LunarLanderContinuous
00:00:00 [INFO] reward_threshold: 200
00:00:00 [INFO] nondeterministic: False
00:00:00 [INFO] max_episode_steps: 1000
00:00:00 [INFO] _kwargs: {}
00:00:00 [INFO] _env_name: LunarLanderContinuous


In [3]:
class ClosedFormAgent:
    def __init__(self, _):
        pass

    def reset(self, mode=None):
        pass

    def step(self, observation, reward, termination):
        x, y, v_x, v_y, angle, v_angle, contact_left, contact_right = observation

        if contact_left or contact_right:  # legs have contact
            f_y = -10. * v_y - 1.
            f_angle = 0.
        else:
            f_y = 5.5 * np.abs(x) - 10. * y - 10. * v_y - 1.
            f_angle = -np.clip(5. * x + 10. * v_x, -4, 4) + 10. * angle + 20. \
                    * v_angle

        action = np.array([f_y, f_angle])
        return action

    def close(self):
        pass


agent = ClosedFormAgent(env)

In [4]:
def play_episode(env, agent, seed=None, mode=None, render=False):
    observation = env.reset(seed=seed)
    reward, termination, truncation = 0., False, False
    agent.reset(mode=mode)
    episode_reward, elapsed_steps = 0., 0
    while True:
        action = agent.step(observation, reward, termination)
        if render:
            env.render()
        if termination or truncation:
            break
        observation, reward, termination, truncation, _ = env.step(action)
        episode_reward += reward
        elapsed_steps += 1
    agent.close()
    return episode_reward, elapsed_steps


logging.info('==== test ====')
episode_rewards = []
for episode in range(100):
    episode_reward, elapsed_steps = play_episode(env, agent)
    episode_rewards.append(episode_reward)
    logging.info('test episode %d: reward = %.2f, steps = %d',
            episode, episode_reward, elapsed_steps)
logging.info('average episode reward = %.2f ± %.2f',
        np.mean(episode_rewards), np.std(episode_rewards))

00:00:00 [INFO] ==== test ====
00:00:00 [INFO] test episode 0: reward = 250.94, steps = 172
00:00:00 [INFO] test episode 1: reward = 263.77, steps = 186
00:00:00 [INFO] test episode 2: reward = 305.75, steps = 202
00:00:00 [INFO] test episode 3: reward = 300.88, steps = 241
00:00:00 [INFO] test episode 4: reward = 303.26, steps = 227
00:00:00 [INFO] test episode 5: reward = 287.23, steps = 205
00:00:00 [INFO] test episode 6: reward = 303.74, steps = 227
00:00:01 [INFO] test episode 7: reward = 299.80, steps = 231
00:00:01 [INFO] test episode 8: reward = 257.93, steps = 181
00:00:01 [INFO] test episode 9: reward = 292.88, steps = 207
00:00:01 [INFO] test episode 10: reward = 292.44, steps = 177
00:00:01 [INFO] test episode 11: reward = 307.51, steps = 237
00:00:01 [INFO] test episode 12: reward = 283.18, steps = 211
00:00:01 [INFO] test episode 13: reward = 300.59, steps = 189
00:00:01 [INFO] test episode 14: reward = 275.73, steps = 187
00:00:01 [INFO] test episode 15: reward = 295.38,

In [5]:
env.close()